### Simpler Model: 1D Convolutional Neural Network (CNN)

To complement the baseline STFT + Conv2D + RNN model, we implemented a lightweight 1D CNN that operates directly on raw ECG signals. This model is more straightforward, easier to train, and more stable in early training phases.

#### Architecture Overview
The model consists of a stack of three 1D convolutional blocks followed by a global average pooling and a fully connected classifier:
- Conv1d → BatchNorm1d → ReLU → MaxPool1d
- Conv1d → BatchNorm1d → ReLU → MaxPool1d
- Conv1d → BatchNorm1d → ReLU → AdaptiveAvgPool1d
- Linear → output logits for 4 classes

Each raw ECG signal is zero-padded to the length of the longest signal in the batch to ensure compatibility with batch processing.

#### Why 1D CNN?
1D CNNs are well-suited for time series tasks like ECG classification due to their ability to extract local patterns (e.g., QRS complexes) efficiently across the signal. Unlike the baseline, this model avoids frequency-domain transformation and recurrent layers, resulting in:
- Faster training
- Lower complexity
- Competitive performance, especially with imbalanced data

In [4]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [5]:
import numpy as np
import torch
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight
from src.ecg_dataset import ECGDataset, prep_batch
from src.cnn1d import CNN1DModel
from src.train import train_model
from src.parser import read_zip_binary

# load data (same as before)
train_idx = np.load("../data/train_idx.npy")
val_idx = np.load("../data/val_idx.npy")

X_train = read_zip_binary("../data/X_train.zip")
y_train = pd.read_csv("../data/y_train.csv", header=None)
y_train.columns = ["y"]

train_dataset = ECGDataset(X_train, y_train, indices=train_idx)
val_dataset = ECGDataset(X_train, y_train, indices=val_idx)

# prepare data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=prep_batch)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=prep_batch)

# device is cpu by default but added this line (instead of hardcoding cpu string) since you may be using cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# weights are added to counter class imbalance
# idea is basically penalizing the rare classes more in order to classify them correctly
weights = compute_class_weight(class_weight="balanced", classes=np.array([0, 1, 2, 3]), y=y_train["y"])

weights = torch.tensor(weights, dtype=torch.float32).to(device)
loss_fn = nn.CrossEntropyLoss(weight=weights)

# load the 1d cnn model into device -- similar practices as always
model = CNN1DModel(n_classes=4).to(device)
# TODO : maybe we can work more on learning rate and find a good balance between lr and # of epochs
optimizer = torch.optim.Adam(model.parameters(), lr = 2e-3)

# enter the training loop -- train_model returns the best model in the end
model = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    device=device,
    num_epochs=50
)

Epoch 01 | Time: 132.1s
  Train Loss: 1.3753 | Acc: 0.2444 | F1: 0.1585
  Val   Loss: 1.3518 | Acc: 0.2581 | F1: 0.1736
Epoch 02 | Time: 132.1s
  Train Loss: 1.3410 | Acc: 0.2764 | F1: 0.2349
  Val   Loss: 1.3181 | Acc: 0.3107 | F1: 0.2855
Epoch 03 | Time: 132.7s
  Train Loss: 1.3010 | Acc: 0.3700 | F1: 0.3079
  Val   Loss: 1.2716 | Acc: 0.2597 | F1: 0.2430
Epoch 04 | Time: 131.8s
  Train Loss: 1.2518 | Acc: 0.3073 | F1: 0.2820
  Val   Loss: 1.2195 | Acc: 0.2314 | F1: 0.2336
Epoch 05 | Time: 132.7s
  Train Loss: 1.2090 | Acc: 0.3761 | F1: 0.3048
  Val   Loss: 1.1901 | Acc: 0.3875 | F1: 0.3117
Epoch 06 | Time: 132.2s
  Train Loss: 1.1728 | Acc: 0.3399 | F1: 0.3008
  Val   Loss: 1.1635 | Acc: 0.4992 | F1: 0.3131
Epoch 07 | Time: 135.4s
  Train Loss: 1.1489 | Acc: 0.4062 | F1: 0.3274
  Val   Loss: 1.1350 | Acc: 0.4094 | F1: 0.3438
Epoch 08 | Time: 132.8s
  Train Loss: 1.1270 | Acc: 0.4117 | F1: 0.3352
  Val   Loss: 1.1522 | Acc: 0.5324 | F1: 0.3362
Epoch 09 | Time: 132.2s
  Train Loss: 1.